# MSG-GAN: Multi-Scale Gradients for Generative Adversarial Networks
https://arxiv.org/pdf/1903.06048.pdf

Implementation by Deniz A. ACAR, Yavuz DURMAZKESER

<p style="text-align: justify">Here we present the implementation of the Multi-Scale Gradient Generative Adversarial Network (MSG-GAN) a simple but effective technique for addressing GAN training problems. This technique provides a stable approach for high resolution image synthesis, and serves as an alternative to the commonly used progressive growing technique (ProGAN [2] and StyleGAN [3]).</p>

The authors claim that `"MSG-GAN converges stably on a variety of image datasets of different sizes, resolutions and domains, as well as different types of loss functions and architectures, all with the same set of fixed hyperparameters."`

***

### Motivation

<p style="text-align: justify">Arjovsky and Bottou [4] pointed out that one of the reasons for the training instability of GANs is due to the
passage of random (uninformative) gradients from the discriminator to the generator when there is insubstantial overlap between the supports of the real and fake distributions.</p>

<p style="text-align: justify">Peng et al. [5] proposed a mutual information bottleneck between input images and the discriminator’s deepest representation of those input images called the variational discriminator bottleneck (VDB) [6], and Karras et al.[7] proposed a progressive growing technique to add continually increasing resolution layers. The VDB solution forces the discriminator to focus only on the most discerning features of the images for classification, which can be viewed as an adaptive variant of instance noise. The progressive growing technique tackles the instability problem by training the GAN layer-by-layer by gradually doubling the operating resolution of the generated images. Whenever a new layer is added to the training it is slowly faded in such that the learning of the previous layers are retained. Intuitively, this technique helps with the support overlap problem because it first achieves a good distribution match on lower resolutions, where the data dimensionality is lower, and then partially-initializes (with substantial support overlap between real and fake distributions) higher resolution training with these previously trained weights, focusing on learning finer details.
</p>
<p style="text-align: justify">
While this approach is able to generate state-of-the-art results, it can be hard to train, due to the addition of hyperparameters to be tuned per resolution, including different iteration counts, learning rates (which can be different for the Generator and Discriminator [8]) and the fade-in iterations. In addition, a concurrent submission [9] discovered that it leads to phase artifacts where certain generated features are attached to specific spatial locations. Hence the main motivation in this paper lies in addressing these problems by providing a simpler alternative that leads to high quality results and stable training.
</p>



The idea behind MSG-GAN is presented below:

<img src="doc/schematic_1.png">

Architecture of MSG-GAN, shown here on the base model proposed in ProGANs which is implemented in this project.

***

### Multi-Scale Gradient GAN

#### Generator Architecture
***
Let the initial block of the generator function $g_{gen} = A_{begin}$ where $A_{begin} \in \mathbb{R}^{4 \times4\times512}$ contains [4x4x512] dimensional activations. Let $g^i$ be a generic function which acts as the basic generator block where in this implementation consists of an upsampling operation followed by two conv layers.

<img src="doc/generator_eqn.png" style="width: 450px;">

where $c_i$ is the number of channels in the $i^{th}$ intermediate activations of the generator. The full generator is as follows:

<img src="doc/full_generator.png" style="width: 700px;">

Where $z~\mathcal{N}(0, \mathbb{I})$.

The generator architecture is presented below:

<img src="doc/generator_schema.png" style="width: 300px;">

***

##### Generator output for different stages

The r is defined such that it generates the output RGB at different stages of the generator where its output corresponds to different downsampled versions of the final output image. It is simply a $(1\times1)$ convolution which converts the intermediate convolutional activation volume into images.

<img src="doc/toRGB.png" style="width: 500px;">

In other words, $o_i$ is an image synthesized from the output of the $i^{th}$ intermediate layer of the generator $a_i$ similar to the idea behind progressive growing.

***

#### Discriminator Architecture

Because the discriminator’s final critic loss is a function of not only the final output of the generator y', but also the intermediate outputs $o_i$, gradients can flow from the intermediate layersof the discriminator to the intermediate layers of the generator.

Here all components of the the discriminator are denoted by `"d"`. 
The final layer of the discriminator is called $d_{critic}(z')$ and the function which defines the first layer of the discriminator is denoted by $d^0(y)$ or $d^0(y)$ taking the real image y (true sample) or the highest resolution synthesized image y' (fake sample) as the input. 

Similarly, let $d^j$ represent the intermediate layer function of the discriminator. Note that i and j are always related to each other as j = k−i. Thus, the output activation volume $a'_j$ of any $j^{th}$ intermediate layer of the discriminator is defined as:

<img src="doc/disc_eqn.png" style="width: 450px;">

where $\phi$ is a function used to combine the output oi of the $i^{th}$ intermediate layer of the generator (or correspondingly downsampled version of the highest resolution real image y) with the corresponding output of the (j − 1)th intermediate layer in the discriminator.

There are three different varients of the $\phi$ which are as follows:

<img src="doc/phi.png" style="width: 400px;">

where, r' is yet another (1x1) convolution operation similar to r and [;] is a simple channelwise concatenation operation.

The full discriminator is as follows:

<img src="doc/discriminator_full.png" style="width: 700px;">

The discriminator architecture is represented below:

<img src="doc/disc_architecture.png">

***

#### Loss Function

<p style="text-align: justify">They used two different loss functions for the $d_{critic}$ function namely, WGAN-GP [10] which wasused by ProGAN [2] and Non-saturating GAN loss with 1-sided GP [11] which was used by StyleGAN [3]. One thing to note that is the fact that the discriminator is now a function of multiple input images generated by the generator, thus gradient penalty is modified to be the average of the penalties over each input.

***
### Implementation Notes

There two different architectures discussed in the paper in accordance with the technique presented here. We have selected ProGANs based architecture to implement for the course project which was also in accordance with our selected qualitative and quantitative results. We have consulted with the instructor of the course and decided to focus on the aforementioned architecture. As the loss function of ProGANs is WGAN-GP is implemented here.    

#Quantitative Results

We aimed to reproduce the results in Table 3, which is in page 8, of the paper, as quantitative results. Results published by the authors are given in the figure below.

<img src="doc/table3.png">

## Testing Inception Score Implementation

In [ ]:
from src.inception_score import Inception_Score
from src.data_loader import DataLoader
import torch

# load real cifar10 images
cifar_dataloader = DataLoader("CIFAR10", batch_size=60000, num_blocks=4, device='cuda')
cifar_images = cifar_dataloader.load_images()[3]

# initialize inception scorer
IS = Inception_Score('cuda')

# compute inception score
real_images_std, real_images_mean = IS(cifar_images)

print("Inception Score of Real CIFAR10 Images (Results in First Row of Table 3): ")
print("\tmean=%.2f, std=%.2f" % (real_images_mean, real_images_std))


Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth



Inception Score of Real CIFAR10 Images (Results in First Row of Table 3): 
	mean=9.45, std=0.13


## Inception Scores of Our Models

In [ ]:
learning_rates = []

for lr in learning_rates:
    # load model
    # generate 50k random images using the model
    # give the generated images to IS as a tensor
    # print results